# ex2 - IsolationForest + XGBoost（使用 %pip 安裝套件）

In [8]:
%pip install kagglehub xgboost

PermissionError: [Errno 1] Operation not permitted

In [ ]:
import kagglehub
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

In [ ]:
# 載入資料
path = kagglehub.dataset_download('mlg-ulb/creditcardfraud')
data = pd.read_csv(f"{path}/creditcard.csv")
data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))
data = data.drop(['Time'], axis=1)

In [ ]:
# Isolation Forest 異常偵測
iso_forest = IsolationForest(n_estimators=100, contamination=0.001, random_state=42)
iso_pred = iso_forest.fit_predict(data.drop(columns=['Class']))
data['Anomaly'] = (iso_pred == -1).astype(int)

In [ ]:
# 分割資料集並轉成 numpy
X = data.drop(columns=['Class'])
y = data['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

X_train = X_train.fillna(0).values
X_test = X_test.fillna(0).values
y_train = y_train.astype(int).values
y_test = y_test.astype(int).values

In [ ]:
# 使用 XGBoost 訓練並評估
model = XGBClassifier(n_estimators=100, max_depth=5, use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))